И на конец-то переходим к 5ой завершающей модели это Neural Network(MLP) простейшая нейросеть для классического ML, попробуем что бы потом спокойно перейти к дл.
Как она работает, как и все нейронные сети есть скрытые и нескрытые слои. Не скрытый слой принимает на вход данные и делает вывод.
Скрытые слои- Один или несколько слоев с нелинейными функциями активации (например, ReLU или tanh).

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import accuracy_score, f1_score
import time
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
split_idx = int(0.8 * len(train))
test = train.iloc[split_idx:]
train = train.iloc[:split_idx]

X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test.drop('Fertilizer Name', axis=1)
y_test = test['Fertilizer Name']


from sklearn.preprocessing import OneHotEncoder
cat_cols = ['Soil Type', 'Crop Type']
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[cat_cols]).toarray()
feature_names = encoder.get_feature_names_out(cat_cols)
X_test_encoded = encoder.transform(X_test[cat_cols]).toarray()

from sklearn.preprocessing import StandardScaler
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

X_train_processed = np.hstack([X_train_scaled, X_train_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_encoded])

results = {}

In [4]:
print("\nОбучение Neural Network (MLP)...")
start_time = time.time()
nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
nn.fit(X_train_processed, y_train)
y_pred = nn.predict(X_test_processed)
results["Neural Network"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}
print(f"Завершено за {time.time() - start_time:.2f} сек. F1: {results['Neural Network']['F1 Score']:.4f}")


Обучение Neural Network (MLP)...
Завершено за 61.26 сек. F1: 0.1510


давайте попробуем подобрать гипер параметры для данной модели как всегда опишем что мы подбираем

hidden_layer_sizes-Размеры скрытых слоев

activation-функция активатор не стал выбирать между relu и другими практика показывает, что это не имеет смысла

alpha-параметры регуляризации

learning_rate_init- скорость обучения

mlp__batch_size-колво данных в одном заходе

Так как наша целеваая переменная -- строки, то пришлось использовать labelencoder, чтобы модель была применима.

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import numpy as np

print("\nИсправленный подбор гиперпараметров с кодированием меток...")
start_time = time.time()


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

if np.isnan(X_train_processed).any() or np.isinf(X_train_processed).any():
    X_train_processed = np.nan_to_num(X_train_processed)
    X_test_processed = np.nan_to_num(X_test_processed)


pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(
        max_iter=1000,
        random_state=42,
        early_stopping=True,
        n_iter_no_change=10
    ))
])


param_grid = {
    'mlp__hidden_layer_sizes': [
        (50,),
        (75,),
        (50, 30),
        (70, 50)
    ],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__alpha': [0.001, 0.01],
    'mlp__batch_size': [32, 64]
}


search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=15,
    cv=3,
    scoring='f1_weighted',
    n_jobs=-1,
    random_state=42,
    verbose=1
)


search.fit(X_train_processed, y_train_encoded)


best_nn = search.best_estimator_
y_pred_encoded = best_nn.predict(X_test_processed)
y_pred = label_encoder.inverse_transform(y_pred_encoded)


results["Neural Network (Tuned)"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='weighted'),
    'Recall': recall_score(y_test, y_pred, average='weighted'),
    'F1 Score': f1_score(y_test, y_pred, average='weighted'),
    'Best Params': search.best_params_
}

print(f"Завершено за {time.time() - start_time:.2f} сек.")
print(f"Лучшие параметры: {search.best_params_}")
print(f"Accuracy: {results['Neural Network (Tuned)']['Accuracy']:.4f}")
print(f"Precision: {results['Neural Network (Tuned)']['Precision']:.4f}")
print(f"Recall: {results['Neural Network (Tuned)']['Recall']:.4f}")
print(f"F1 Score: {results['Neural Network (Tuned)']['F1 Score']:.4f}")



Исправленный подбор гиперпараметров с кодированием меток...
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Завершено за 1796.11 сек.
Лучшие параметры: {'mlp__hidden_layer_sizes': (50,), 'mlp__batch_size': 64, 'mlp__alpha': 0.001, 'mlp__activation': 'relu'}
Accuracy: 0.1639
Precision: 0.1604
Recall: 0.1639
F1 Score: 0.1456
